# scaredy-rat uses raw data from EthoVision to analyze behavior over time. 

#### Notes for workarounds:

 - in the **animal_read** function, the indexes are specific to the ethovision output. sometimes there are more or fewer rows in the header, so the numbers will have to be changed. if that function throws an error, this may be the issue. for reference, the animal ID is row 33 on the excel file, so the corresponding index is **df.iloc[31]**. if the animal ID were on row 34, you would change the index to **df.iloc[32]**.
 - in the same vein, the header row will need to be readjusted: if header row is on row 36 in the data file, then **skiprows = 35**.

In [ ]:
# notebook formatting and loading settings - keep same

%matplotlib inline
%reload_ext autoreload
%autoreload 2

In [ ]:
# imports - keep same

import os, sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML
import math
import time

sys.path.append('../src')
import scaredyrat as sr

# Change variables in this cell:

_inpath_ should stay constant as the data should all be in one folder. _outpath_ is wherever you want the analyzed files to go. Each raw .xlsx file has four sheets, one for each animal. Change _sheetname_ every run, and change _filename_ once all the sheets have been run.

In [ ]:
## Be sure to put r before the path
## Easiest to go to correct folder, then copy/paste its path
inpath = r'C:\Users\Anna\Documents\Shansky Lab\Mindlin\Mindlin FC Raw Data'

## Make sure the folder exists already.
## The file does not need to exist; it will be created.
outpath = r'C:\Users\Anna\Documents\Shansky Lab\170808 mindlin analysis'

filename = 'Raw data-Mindlin Project Fear Conditioning-Trial    17 (1).xlsx' ## include .xlsx
sheetname = 'Track-Arena 4-Subject 1'

## Function calls to read animal data

In [ ]:
ID,anim = animal_read(filename,sheetname)

tones = sr.find_tones(anim)
pretones = sr.find_pretones(anim)
shocks = sr.find_shock_responses(anim)
pshocks = sr.find_postshocks(anim)

## Function calls to find max velocities

In [ ]:
## The integer (e.g. the 10 in (tones, 10) determines how many maximum velocities to find)
mtone = sr.get_top_vels(tones,10)
mptone = sr.get_top_vels(pretones,10)
mshock = sr.get_top_vels(shocks,5)
mpshock = sr.get_top_vels(pshocks,10)

## Function calls to write max velocities to a csv

In [ ]:
## Saves the data to a file named with current date.
i = time.strftime("%m.%d.%Y")

mtoneoutfile = outpath + '\\scaredy-rat-tone-max-' + ID + '-' + i + '.csv'
mptoneoutfile = outpath + '\\scaredy-rat-pretone-max-' + ID + '-' + i + '.csv'
mshockoutfile = outpath + '\\scaredy-rat-shock-max-' + ID + '-' + i + '.csv'
mpshockoutfile = outpath + '\\scaredy-rat-postshock-max-' + ID + '-' + i + '.csv'

mtone.to_csv(mtoneoutfile)
mptone.to_csv(mptoneoutfile)
mshock.to_csv(mshockoutfile)
mpshock.to_csv(mpshockoutfile)

## Function calls for to find means and meds

In [ ]:
tmeans = sr.get_means(tones,'Tone')
ptmeans = sr.get_means(pretones,'Pre-tone')
smeans = sr.get_means(shocks,'Shock')
pshockmeans = sr.get_means(pshocks,'Post-shock')
allmeans = pd.concat([tmeans, ptmeans, smeans, pshockmeans],axis=1)

tmeds = sr.get_meds(tones,'Tone')
ptmeds = sr.get_meds(pretones,'Pre-tone')
smeds = sr.get_meds(shocks,'Shock')
pshockmeds = sr.get_meds(pshocks,'Post-shock')
allmeds = pd.concat([tmeds, ptmeds, smeds, pshockmeds],axis=1)

## Function calls to write means and meds to a csv

In [ ]:
## Saves the data to a file named with current date.
i = time.strftime("%m.%d.%Y")
meanoutfile = outpath + '\\scaredy-rat-mean-' + ID + '-' + i + '.csv'
medoutfile = outpath + '\\scaredy-rat-med-' + ID + '-' + i + '.csv'

allmeans.to_csv(meanoutfile)
allmeds.to_csv(medoutfile)

## Function calls for plotting

In [ ]:
# Plots are autosaved into outpath folder

vels = pd.DataFrame(anim['Velocity'])

plt.style.use('seaborn-white')
plt.figure(figsize=(16,8),facecolor='white',edgecolor='white')
# Plots main velocity
plt.plot(vels,color='k',linewidth=0.1)

# Loops through tones, plots each one.
i = 1
while i <= 12:
    tone = find_tone_vels(anim,i)
    plt.plot(tone,color='c',linewidth=1)
    i += 1

# Loops through shocks, plots each one.
i = 6
while i <= 12:
    sresponse = find_shock_vels(anim,i)
    plt.plot(sresponse,color='m',linewidth=1)
    i += 1

sns.despine(left=True, bottom=True, right=True)
plt.title(ID)
plt.ylabel('Velocity (cm/s)')
plt.xlabel('Trial time (s)')

fname = outpath + '\\' + ID + '-vel-plot'
plt.savefig(fname, dpi=300)

plt.show()
plt.close()